In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using Tesla K80 GPUs


In [ ]:
! pip install datasets transformers seqeval

In [3]:
task = "ner"
batch_size = 8

# Загрузка данных - Primary Account Documents




In [4]:
!gdown --id 1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ

Downloading...
From: https://drive.google.com/uc?id=1cCXjHX9FAgouF0DWuWPO5qyBHVZDkXTQ
To: /content/NER_PAD_agg_test.csv
2.92MB [00:00, 84.1MB/s]


In [5]:
!gdown --id 1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t

Downloading...
From: https://drive.google.com/uc?id=1RFNBRcly96omdpNtYlEK6O1EuQs3ux7t
To: /content/NER_PAD_agg_train.csv
11.7MB [00:00, 32.0MB/s]


In [6]:
!gdown --id 1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV

Downloading...
From: https://drive.google.com/uc?id=1AnLqXtSyJNBK7YmwuW6L6z2OYhE4RPTV
To: /content/NER_PAD_agg.csv
14.6MB [00:00, 31.3MB/s]


In [7]:
import pandas as pd
data = pd.read_csv('NER_PAD_agg.csv', converters={'tokens': eval, 'ner_tags': eval})

In [ ]:
all_tags = set()
for _, row in data.iterrows():
    for tag in row.ner_tags:
        all_tags.add(tag)
all_tags = sorted(list(all_tags))

In [10]:
tag_to_idx = {t: i for i, t in enumerate(all_tags)}

In [11]:
label_list = {i: t for i, t in enumerate(all_tags)}

In [12]:
train = pd.read_csv('NER_PAD_agg_train.csv', converters={'tokens': eval, 'ner_tags': eval})
test = pd.read_csv('NER_PAD_agg_test.csv', converters={'tokens': eval, 'ner_tags': eval})

In [13]:
train.ner_tags = train.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])
test.ner_tags = test.ner_tags.apply(lambda x : [tag_to_idx[label] for label in x])

In [14]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

In [15]:
from datasets import load_dataset, load_metric

In [ ]:
datasets = load_dataset('pandas', data_files={'train' : 'train.pkl', 'test' : 'test.pkl'})

In [ ]:
datasets

# Токенизатор


In [18]:
from transformers import AutoTokenizer
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [19]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [20]:
label_all_tokens = False

In [21]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (954 > 512). Running this sequence through the model will result in indexing errors


Warning : есть последовательности больше, чем максимально возможная в BERT - 512

In [23]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    3200.000000
mean      454.245000
std       283.854426
min         6.000000
25%       207.000000
50%       405.000000
75%       566.000000
max      4271.000000
dtype: float64

In [24]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count     800.000000
mean      453.612500
std       281.724152
min        47.000000
25%       202.000000
50%       406.000000
75%       566.500000
max      1796.000000
dtype: float64

In [25]:
train_tokenized = tokenized_datasets["train"].to_pandas()
test_tokenized = tokenized_datasets["test"].to_pandas()

In [28]:
CHUNCK_SIZE = 512

def lists_to_chuncks(row):
    n = CHUNCK_SIZE - 2
    new_row = row.copy()
    for key in ['attention_mask', 'input_ids', 'labels', 'token_type_ids']:
        FIRST = new_row[key][0]
        LAST = new_row[key][-1]
        new_row[key] = new_row[key][1:-1]
        new_row[key] = [[FIRST] + list(new_row[key][i:i + n]) + [LAST] for i in range(0, len(new_row[key]), n)]
    return new_row

In [29]:
train_tokenized = train_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
train_tokenized = train_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [30]:
test_tokenized = test_tokenized.drop(columns = ['ner_tags', 'tokens']).apply(lists_to_chuncks, axis=1)
test_tokenized = test_tokenized.set_index(['id']).apply(pd.Series.explode).reset_index()

In [31]:
train_tokenized.to_pickle('train_tokenized.pkl')
test_tokenized.to_pickle('test_tokenized.pkl')

In [32]:
tokenized_datasets = load_dataset('pandas', data_files={'train' : 'train_tokenized.pkl', 'test' : 'test_tokenized.pkl'})

Using custom data configuration default-e8b3fc59bf9f3965


Dataset pandas downloaded and prepared to /root/.cache/huggingface/datasets/pandas/default-e8b3fc59bf9f3965/0.0.0/3547ff13f14ebf30dc8f11ab416dcf7721c18de2eb7e054bf9c80b88e765a791. Subsequent calls will reuse this data.


In [33]:
pd.Series(tokenized_datasets["train"]['input_ids']).apply(lambda x : len(x)).describe()

count    4415.000000
mean      329.787995
std       162.203175
min         3.000000
25%       184.000000
50%       358.000000
75%       512.000000
max       512.000000
dtype: float64

In [34]:
pd.Series(tokenized_datasets["test"]['input_ids']).apply(lambda x : len(x)).describe()

count    1090.000000
mean      333.458716
std       159.904559
min         3.000000
25%       183.000000
50%       369.500000
75%       512.000000
max       512.000000
dtype: float64

Теперь все последовательности нужной длины

# Модель

# Обучение


In [35]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Warning : модель надо обучить на наших данных

In [36]:
args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [37]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [38]:
metric = load_metric("seqeval")

In [39]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [40]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [41]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.124900,0.049442,0.797494,0.803350,0.800412,0.985790
2,0.045000,0.039823,0.823322,0.858651,0.840616,0.988246
3,0.034200,0.039194,0.839460,0.869894,0.854406,0.988972


TrainOutput(global_step=1656, training_loss=0.06432934141389414, metrics={'train_runtime': 2960.4441, 'train_samples_per_second': 0.559, 'total_flos': 675171562108416.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 1439895552, 'init_mem_gpu_alloc_delta': 709382656, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -736489472, 'train_mem_gpu_alloc_delta': 2197179392, 'train_mem_cpu_peaked_delta': 774184960, 'train_mem_gpu_peaked_delta': 6528750592})

# Сохранение модели

In [50]:
import pickle
with open("PAD_BERT[bert_base_multilingual_cased].pkl", 'wb') as f:
    pickle.dump(model, f)

# Валидация

In [42]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_accuracy': 0.9889720634832783,
 'eval_f1': 0.8544061302681992,
 'eval_loss': 0.039194196462631226,
 'eval_mem_cpu_alloc_delta': 212992,
 'eval_mem_cpu_peaked_delta': 3338240,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 448418304,
 'eval_precision': 0.8394596988485385,
 'eval_recall': 0.869894446994034,
 'eval_runtime': 79.3108,
 'eval_samples_per_second': 13.743}

In [43]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'DOCAGRDATE': {'f1': 0.7550432276657061,
  'number': 151,
  'precision': 0.6683673469387755,
  'recall': 0.8675496688741722},
 'DOCAGRNUM': {'f1': 0.6277777777777778,
  'number': 179,
  'precision': 0.6243093922651933,
  'recall': 0.6312849162011173},
 'DOCAMOUNT': {'f1': 0.8629760502552021,
  'number': 1244,
  'precision': 0.8434382194934766,
  'recall': 0.8834405144694534},
 'DOCCPTY': {'f1': 0.8580294802172227,
  'number': 596,
  'precision': 0.797979797979798,
  'recall': 0.9278523489932886},
 'DOCCPTYINN': {'f1': 0.935374149659864,
  'number': 282,
  'precision': 0.8986928104575164,
  'recall': 0.975177304964539},
 'DOCCUSTOMER': {'f1': 0.9358669833729216,
  'number': 610,
  'precision': 0.9050535987748851,
  'recall': 0.9688524590163935},
 'DOCDATE': {'f1': 0.8882309400444116,
  'number': 645,
  'precision': 0.8498583569405099,
  'recall': 0.9302325581395349},
 'DOCNUM': {'f1': 0.7599645704162976,
  'number': 651,
  'precision': 0.897489539748954,
  'recall': 0.6589861751152074}

In [44]:
pd.DataFrame.from_dict(results).transpose().iloc[:-4, :-1]

,precision,recall,f1
DOCAGRDATE,0.668367,0.867550,0.755043
DOCAGRNUM,0.624309,0.631285,0.627778
DOCAMOUNT,0.843438,0.883441,0.862976
DOCCPTY,0.797980,0.927852,0.858029
DOCCPTYINN,0.898693,0.975177,0.935374
DOCCUSTOMER,0.905054,0.968852,0.935867
DOCDATE,0.849858,0.930233,0.888231
DOCNUM,0.897490,0.658986,0.759965


In [ ]:
!pip install sklearn_crfsuite

In [47]:
from sklearn_crfsuite import metrics
print(metrics.flat_classification_report(true_labels, true_predictions, labels=all_tags[:-1], digits=3))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


               precision    recall  f1-score   support

 B-DOCAGRDATE      0.712     0.907     0.798       150
  B-DOCAGRNUM      0.663     0.670     0.667       179
  B-DOCAMOUNT      0.863     0.891     0.877      1243
    B-DOCCPTY      0.837     0.931     0.881       595
 B-DOCCPTYINN      0.899     0.975     0.935       282
B-DOCCUSTOMER      0.928     0.972     0.949       609
    B-DOCDATE      0.886     0.949     0.916       645
     B-DOCNUM      0.897     0.659     0.760       651
 I-DOCAGRDATE      0.658     0.852     0.743        88
  I-DOCAGRNUM      0.000     0.000     0.000         7
  I-DOCAMOUNT      0.821     0.885     0.852       620
    I-DOCCPTY      0.818     0.954     0.881       564
I-DOCCUSTOMER      0.902     0.955     0.927       462
    I-DOCDATE      0.898     0.960     0.928       605
     I-DOCNUM      0.000     0.000     0.000         0

    micro avg      0.858     0.897     0.877      6700
    macro avg      0.719     0.771     0.741      6700
 weighte